In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
import numpy as np
 
%matplotlib inline

#Load Data with pandas, and parse the first column into datetime
train=pd.read_csv('train.csv', parse_dates = ['Dates'])
test=pd.read_csv('test.csv', parse_dates = ['Dates'])

train.head()

/home/jsutker/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [3]:
#Convert crime labels to numbers
le_crime = preprocessing.LabelEncoder()
crime = le_crime.fit_transform(train.Category)
 
#Get binarized weekdays, districts, and hours.
days = pd.get_dummies(train.DayOfWeek)
district = pd.get_dummies(train.PdDistrict)
hour = train.Dates.dt.hour
hour = pd.get_dummies(hour) 
 
#Build new array
train_data = pd.concat([hour, days, district], axis=1)
train_data['crime']=crime
 
#Repeat for test data
days = pd.get_dummies(test.DayOfWeek)
district = pd.get_dummies(test.PdDistrict)
 
hour = test.Dates.dt.hour
hour = pd.get_dummies(hour) 
 
test_data = pd.concat([hour, days, district], axis=1)
 
training, validation = train_test_split(train_data, train_size=.60)

print train_data.head()

   0  1  2  3  4  5  6  7  8  9  ...    CENTRAL  INGLESIDE  MISSION  NORTHERN  \
0  0  0  0  0  0  0  0  0  0  0  ...          0          0        0         1   
1  0  0  0  0  0  0  0  0  0  0  ...          0          0        0         1   
2  0  0  0  0  0  0  0  0  0  0  ...          0          0        0         1   
3  0  0  0  0  0  0  0  0  0  0  ...          0          0        0         1   
4  0  0  0  0  0  0  0  0  0  0  ...          0          0        0         0   

   PARK  RICHMOND  SOUTHERN  TARAVAL  TENDERLOIN  crime  
0     0         0         0        0           0     37  
1     0         0         0        0           0     21  
2     0         0         0        0           0     21  
3     0         0         0        0           0     16  
4     1         0         0        0           0     16  

[5 rows x 42 columns]


In [4]:
features = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
 'Wednesday', 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION',
 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN']

features2 = [x for x in range(0,24)]
features = features + features2

training, validation = train_test_split(train_data, train_size=.60)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(training[features], training['crime'])
predicted = np.array(model.predict_proba(validation[features]))
log_loss(validation['crime'], predicted)

3.0185449165351894

In [11]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(training[features], training['crime'])
predicted = np.array(model.predict_proba(validation))
log_loss(validation['crime'], predicted)

AttributeError: 'LinearRegression' object has no attribute 'predict_proba'

In [9]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=6)
model.fit(training[features], training['crime'])
predicted = np.array(model.predict_proba(test_data[features]))

#Write results
result=pd.DataFrame(predicted, columns=le_crime.classes_)
result.to_csv('DTtestResult.csv', index = True, index_label = 'Id' )

In [6]:
len(result)

351220